# QA files for regression prepration

In [40]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import re
import string 

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

# ignore warning
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings('ignore')

# for stock price
import yfinance as yf
from yahoofinancials import YahooFinancials


[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
# read the csv file 
topic_modelling_df_QA = pd.read_csv('./main_df_output/QA12_topic_modelling_df.csv')

# Define Functions

In [42]:
# clean text
def process_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    #text = text.str
    text = str(text)
    text = re.sub(r'\$\w*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'#', '', text)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    text_tokens = tokenizer.tokenize(text)

    text_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)
            
    sentence = ' '.join(text_clean)
    
    return sentence

In [43]:
# calculate the stock price change
def stock_price_change(stock_list, start_date, end_date):
    # function for find price change
    def price_change(aapl_df, n_day):
        before = []
        after = []
        for i in range(len(aapl_df)):
            # price on that date
            prc_tdy = aapl_df['Close'][i]

            if ((i >= n_day) & 
                (i < (len(aapl_df)-n_day))):
                # price change before n_day
                prc_before = aapl_df['Close'][i-n_day]
                prc_change_before = (prc_tdy - prc_before)/prc_before
                # price change after n_day
                prc_after = aapl_df['Close'][i+n_day]
                prc_change_after = (prc_after - prc_tdy)/prc_tdy
            else:
                prc_change_before = 0
                prc_change_after = 0
            # append into list
            before.append(prc_change_before)
            after.append(prc_change_after)
        return before, after
        
    # tickers and its closing stock price
    stock_df = pd.DataFrame()
    for i in stock_list:
        aapl_df = yf.download(i, 
                        start= start_date,  # start='2010-12-01', # start 1 month before
                        end = end_date,   # end='2022-01-30', # end 1 month later
                        progress=False,)
        aapl_df = aapl_df.reset_index(drop=False)
        # add ticker name
        aapl_df['ticker']=i
        # add price change
        # aapl_df['D0'] = aapl_df['Close']
        aapl_df['D-1'],aapl_df['D+1'] = price_change(aapl_df, 1)
        aapl_df['D-2'],aapl_df['D+2'] = price_change(aapl_df, 2)
        aapl_df['D-3'],aapl_df['D+3'] = price_change(aapl_df, 3)
        aapl_df['D-5'],aapl_df['D+5'] = price_change(aapl_df, 5)
        aapl_df['D-10'],aapl_df['D+10'] = price_change(aapl_df, 10)
        aapl_df['D-15'],aapl_df['D+15'] = price_change(aapl_df, 15)
        # append into one dataframe 
        stock_df = stock_df.append(aapl_df)
        
    # drop redundancy columns
    stock_price_df = stock_df.drop(columns=['Open', 'High','Low','Adj Close','Volume'])
    stock_price_df = stock_price_df.rename(columns={'Date': 'date'})
    # drop duplicated rows
    stock_price_df = stock_price_df.drop_duplicates()
    return stock_price_df

# Clean text and splitting to sentence

In [44]:
df = topic_modelling_df_QA.drop(['participants','idx','company_paticipants_yes','other_paticipants_yes',
                'paraghrph_noun', 'word_count', 'char_count', 'sentence_count',
                'avg_word_length', 'avg_sentence_length'], axis = 1)
df = df.rename(columns = {'paraghrph':'paraghraph','paraghrph_clean':'paraghraph_clean'})
df = df.sort_values(by=['file_name'])
df = df.reset_index(drop=True)

# splotting into the sentences
df['paraghraph'] = df['paraghraph'].apply(lambda r: r.replace("Mr. ","Mr."))
df['sentence'] = df['paraghraph'].apply(lambda r: r.split(". "))
df = df.explode('sentence')

# Apply the function to clean the text
df['clean_text'] = df['sentence'].apply(lambda r: process_text(r))
df.head(5)

df.head(5)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,sentence,clean_text
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,0.279804,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,"James Quin, Citigroup",jame quin citigroup
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,0.279804,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,James Quin 3878205 Three numbers questions p...,jame quin 3878205 three number question pleas
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,0.279804,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,The first one is,first one
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,0.279804,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,And I know this is something that's come up on...,know someth that' come previou call
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,0.279804,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,"But I'm going to ask you anyway, is just could...",i'm go ask anyway could give us sens prior rel...


# Weighted topic Probability by files

In [45]:
topic_df = df.copy()
# cal len paragraph
topic_df['len_para'] = topic_df['paraghraph'].apply(lambda r: len(r))

In [46]:
cal_len_docs = topic_df[['file_name','len_para']]
# Use GroupBy() to compute the sum of the document
cal_len_docs = cal_len_docs.groupby('file_name').sum()
cal_len_docs = cal_len_docs.rename(columns={"len_para":"len_docs"})
cal_len_docs = cal_len_docs.reset_index(drop=False)
cal_len_docs.head(5)

,file_name,len_docs
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2003760
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,494566
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,1775115
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,1178324
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,1372839


In [47]:
# calculate weighted average for topic probability
# -----> merge the cal_len_docs
merge_df = cal_len_docs.merge(topic_df, how='inner', on='file_name')
# -----> caculate the weight of the sentiment
merge_df['weighted']=merge_df['len_para']/merge_df['len_docs']


In [48]:
merge_df.head(1)

,file_name,len_docs,date,company_name,paraghraph,paraghraph_clean,token,topic_1,topic_2,topic_3,...,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,sentence,clean_text,len_para,weighted
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2003760,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,...,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,"James Quin, Citigroup",jame quin citigroup,7785,0.003885


In [49]:
# -----> give the weighted average of the probability based on the paragraph length
for i in range (1,13):
    merge_df['topic_'+str(i)] = merge_df['topic_'+str(i)]*merge_df['weighted']
# -----> sum WA sentiment of each files
topic_df = merge_df.copy()
topic_df = topic_df.groupby(['file_name','date','company_name']).sum().reset_index(drop=False)
# drop the column: 'len_docs', 'len_para', 'weighted'
topic_df = topic_df.drop(['len_docs','len_para','weighted'], axis = 1)
topic_df

,file_name,date,company_name,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,0.030735,0.009612,0.035099,0.005009,0.363800,0.034760,0.015954,0.189425,0.136196,0.105219,0.071154,0.003037
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,0.023797,0.025898,0.115187,0.145184,0.026819,0.130131,0.011664,0.164773,0.170240,0.150919,0.011495,0.023892
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,0.109908,0.006471,0.042130,0.006111,0.054999,0.194784,0.162813,0.088680,0.226127,0.031044,0.071282,0.005651
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,0.192408,0.163730,0.014077,0.022660,0.045273,0.162033,0.015460,0.015536,0.031294,0.147962,0.043274,0.146293
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,2011-02-09,Sampo Oyj-,0.086062,0.015130,0.009820,0.023597,0.179287,0.068430,0.018734,0.112383,0.074864,0.391000,0.010428,0.010265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,0.015220,0.092127,0.084452,0.004813,0.014671,0.201813,0.006813,0.110140,0.185415,0.090803,0.006715,0.187019
1430,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,0.005090,0.043759,0.021687,0.018146,0.141261,0.219800,0.072607,0.202345,0.008600,0.224878,0.037405,0.004423
1431,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,0.013193,0.025080,0.010250,0.056805,0.203925,0.141213,0.157895,0.041687,0.051532,0.116239,0.167455,0.014726
1432,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,0.056249,0.067623,0.020717,0.011767,0.066955,0.029426,0.047228,0.071607,0.013334,0.151718,0.008727,0.454650


# Weighted Sentiment Score by files

In [50]:
import pickle
sentiment = pickle.load(open('sentiment_model.pkl', 'rb'))
df['sentiment'] = sentiment.predict(df['clean_text'])
df.head(3)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,topic_1,topic_2,topic_3,topic_4,...,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,sentence,clean_text,sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,"James Quin, Citigroup",jame quin citigroup,1
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,James Quin 3878205 Three numbers questions p...,jame quin 3878205 three number question pleas,1
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.005235,0.003578,0.101412,0.361265,0.005406,0.101151,0.00235,The first one is,first one,1


In [51]:
# Weight of each sentimentc
# cal len paragraph
df['len_para'] = df['paraghraph'].apply(lambda r: len(r))
# cal len sentence 
df['len_sent'] = df['sentence'].apply(lambda r: len(r))
df.head(3)

,file_name,date,company_name,paraghraph,paraghraph_clean,token,topic_1,topic_2,topic_3,topic_4,...,topic_8,topic_9,topic_10,topic_11,topic_12,sentence,clean_text,sentiment,len_para,len_sent
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.101412,0.361265,0.005406,0.101151,0.00235,"James Quin, Citigroup",jame quin citigroup,1,7785,25
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.101412,0.361265,0.005406,0.101151,0.00235,James Quin 3878205 Three numbers questions p...,jame quin 3878205 three number question pleas,1,7785,51
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,"James Quin, Citigroup. James Quin 3878205...",James Quin Citigroup James Quin Three n...,"['contribution', 'debate', 'phase', 'nonlife',...",0.068571,0.017883,0.050817,0.002528,...,0.101412,0.361265,0.005406,0.101151,0.00235,The first one is,first one,1,7785,16


In [52]:
cal_len_docs = df[['file_name','len_para']]
# Use GroupBy() to compute the sum of the document
cal_len_docs = cal_len_docs.groupby('file_name').sum()
cal_len_docs = cal_len_docs.rename(columns={"len_para":"len_docs"})
cal_len_docs = cal_len_docs.reset_index(drop=False)
cal_len_docs.head(5)

,file_name,len_docs
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2003760
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,494566
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,1775115
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,1178324
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,1372839


In [53]:
# calculate weighted average for sentiment
# -----> merge the cal_len_docs
merge_df = cal_len_docs.merge(df, how='inner', on='file_name')
# -----> caculate the weight of the sentiment
merge_df['weighted']=merge_df['len_sent']/merge_df['len_docs']
# -----> give the weighted average of the sentiment score based on the sentence
merge_df['WA_sentiment'] = merge_df['weighted']*merge_df['sentiment']
# -----> sum WA sentiment of each files
sentiment_df = merge_df[['file_name','date','company_name','WA_sentiment']]
sentiment_df = sentiment_df.groupby(['file_name','date','company_name']).sum().reset_index(drop=False)
sentiment_df

,file_name,date,company_name,WA_sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,0.003787
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,0.014049
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,0.007116
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,0.003145
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,2011-02-09,Sampo Oyj-,0.000196
...,...,...,...,...
1429,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,0.005986
1430,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,0.002872
1431,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,0.005447
1432,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,0.004821


# merge the sentiment_df and topic_df together

In [54]:
# merge the sentiment_df and topic_df together
QA_model_df = topic_df.merge(sentiment_df, how='inner', on=['file_name','date','company_name'])
QA_model_df

,file_name,date,company_name,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,WA_sentiment
0,20110202 _Mapfre_SA_Earnings_Call_SD0000000027...,2011-02-02,Mapfre SA,0.030735,0.009612,0.035099,0.005009,0.363800,0.034760,0.015954,0.189425,0.136196,0.105219,0.071154,0.003037,0.003787
1,20110202_LEGAL_-_GEN_GRP-_Guidance_Call_2011-2...,2011-02-02,LEGAL -,0.023797,0.025898,0.115187,0.145184,0.026819,0.130131,0.011664,0.164773,0.170240,0.150919,0.011495,0.023892,0.014049
2,20110203_Markel_Corp-_Earnings_Call_2011-2-3_S...,2011-02-03,Markel Corp-,0.109908,0.006471,0.042130,0.006111,0.054999,0.194784,0.162813,0.088680,0.226127,0.031044,0.071282,0.005651,0.007116
3,20110209_INTACT_FINANCIAL-_Earnings_Call_2011-...,2011-02-09,INTACT FINANCIAL-,0.192408,0.163730,0.014077,0.022660,0.045273,0.162033,0.015460,0.015536,0.031294,0.147962,0.043274,0.146293,0.003145
4,20110209_Sampo_Oyj-_Earnings_Call_2011-2-9_SD0...,2011-02-09,Sampo Oyj-,0.086062,0.015130,0.009820,0.023597,0.179287,0.068430,0.018734,0.112383,0.074864,0.391000,0.010428,0.010265,0.000196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429,20220310_Sanlam_Ltd-_Earnings_Call_2022-3-10_D...,2022-03-10,Sanlam Ltd-,0.015220,0.092127,0.084452,0.004813,0.014671,0.201813,0.006813,0.110140,0.185415,0.090803,0.006715,0.187019,0.005986
1430,20220323_Poste_Italiane_SpA-_Earnings_Call_202...,2022-03-23,Poste Italiane,0.005090,0.043759,0.021687,0.018146,0.141261,0.219800,0.072607,0.202345,0.008600,0.224878,0.037405,0.004423,0.002872
1431,20220324_Helvetia_Holding_AG-_Earnings_Call_20...,2022-03-24,Helvetia Holding,0.013193,0.025080,0.010250,0.056805,0.203925,0.141213,0.157895,0.041687,0.051532,0.116239,0.167455,0.014726,0.005447
1432,20220426_Tryg_A-S-_Earnings_Call_2022-4-26_DN0...,2022-04-26,Tryg A-S-,0.056249,0.067623,0.020717,0.011767,0.066955,0.029426,0.047228,0.071607,0.013334,0.151718,0.008727,0.454650,0.004821


# Stock Price

In [55]:
# List all yahoo tickers
yahoo_ticker_list = [
    # Motor/Personal
    'ADM.L','DLG.L','SBRE.L','SAGA.L','AGS.BR',
    # Global Commercial
    'ALV.DE','CS.PA','ZURN.SW','G.MI',
    # London Market 
    'BEZ.L','HSX.L','LRE.L',
    # LN Equity 
    'LLOY.L',
    # US - Specialty/P&C/Reinsurance
    'AIG','AXS','TRV','ACGL','RNR','RE','MKL','HIG','ARGO','BRK-B','CB',
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8630.T','8725.T','QBE.AX',
    # Run-off
    'ESGR','FFH.TO','RQIH.L',
    # Life Groups & Retail Life
    'PRU.L','MNG.L','LGEN.L','AV.L','PHNX.L','QLT.L','JUST.L','STJ.L','AGN.AS','DSY.JO','SLM.JO',
    # Other insurers
    'STB.OL','CNP.PA','GJF.OL','PST.MI','NN.AS','TOP.CO','BALN.SW',
    'SAMPO.HE','MAP.MC','TRYG.CO','0RHS.IL','HELN.SW','IFC.TO'
]

# Match it with company names from BoE
Insurer_Names_df = pd.read_excel('./input/Insurer_Names_for_possible_NLP_analysis.xlsx')  
Insurer_Names = Insurer_Names_df.drop(['Unnamed: 0', 'Unnamed: 4'], axis=1)
Insurer_Names = Insurer_Names.drop([0])
Insurer_Names.rename(columns={"Unnamed: 1": "Company", "Unnamed: 2": "bb ticker", "Unnamed: 3":"Group"}, inplace = True)
Insurer_Names['yahoo ticker'] = yahoo_ticker_list
Insurer_Names.head(5)
# Exported as CSV to manually matched with the extracted file names (column 'company_name' in model_df)
# Insurer_Names[['Company']].to_csv('./main_df_output/company_name_match.csv', index = False)

,Company,bb ticker,Group,yahoo ticker
1,ADMIRAL GROUP,ADM LN,Motor/Personal,ADM.L
2,DIRECT LINE INSU,DLG LN,Motor/Personal,DLG.L
3,SABRE INSUR,SBRE LN,Motor/Personal,SBRE.L
4,SAGA PLC,SAGA LN,Motor/Personal,SAGA.L
5,AGEAS,AGS BB,Motor/Personal,AGS.BR


In [56]:
company_name_match = pd.read_csv('./input/company_name_match.csv')
company_name_match.rename(columns={"Company": "company_name"}, inplace = True)
company_name_match

Groups = {
        "ADMIRAL GROUP" : "Motor/Personal", "DIRECT LINE INSU" : "Motor/Personal", 
        "SABRE INSUR" : "Motor/Personal", "SAGA PLC" : "Motor/Personal",
        "AGEAS" : "Motor/Personal",

        "ALLIANZ SE-REG" : "Global Commercial", "AXA" : "Global Commercial", 
        "ZURICH INSURANCE" : "Global Commercial", "GENERALI ASSIC" : "Global Commercial",

        "BEAZLEY PLC" : "London Market", "HISCOX LTD" : "London Market", 
        "LANCASHIRE HOLDI" : "London Market",
        
        "Society of Lloyd’s" : "LN Equity",

        "AMERICAN INTERNA" : "US - Specialty/P&C/Reinsurance", "AXIS CAPITAL" : "US - Specialty/P&C/Reinsurance",
        "TRAVELERS COS IN" : "US - Specialty/P&C/Reinsurance", "ARCH CAPITAL GRP" : "US - Specialty/P&C/Reinsurance",
        "RENAISSANCERE" : "US - Specialty/P&C/Reinsurance", "EVEREST RE GROUP" : "US - Specialty/P&C/Reinsurance",
        "MARKEL CORP" : "US - Specialty/P&C/Reinsurance", "HARTFORD FINL SV" : "US - Specialty/P&C/Reinsurance",
        "ARGO GROUP INTER" : "US - Specialty/P&C/Reinsurance", "BERKSHIRE HATH-B" : "US - Specialty/P&C/Reinsurance",
        "CHUBB LTD" : "US - Specialty/P&C/Reinsurance",

        "SCOR SE" : "European (Re)Insurers", "MUENCHENER RUE-R" : "European (Re)Insurers", 
        "SWISS RE AG" : "European (Re)Insurers", "HANNOVER RUECK S" : "European (Re)Insurers",

        "TOKIO MARINE HD" : "Japanese & Pacific", "SOMPO HOLDINGS I" : "Japanese & Pacific",
        "MS&AD INSURANCE" : "Japanese & Pacific", "QBE INSURANCE" : "Japanese & Pacific",

        "ENSTAR GROUP LTD" : "Run-off", "FAIRFAX FINL HLD" : "Run-off", "RANDALL & QUILTE" : "Run-off",

        "PRUDENTIAL PLC" : "Life Groups & Retail Life", "M&G PLC" : "Life Groups & Retail Life",
        "LEGAL & GEN GRP" : "Life Groups & Retail Life", "AVIVA PLC" : "Life Groups & Retail Life",
        "PHOENIX GROUP HO" : "Life Groups & Retail Life", "QUILTER PLC" : "Life Groups & Retail Life",
        "JUST GROUP" : "Life Groups & Retail Life", "ST JAMES'S PLACE" : "Life Groups & Retail Life",
        "AEGON NV" : "Life Groups & Retail Life", "DISCOVERY LTD" : "Life Groups & Retail Life",
        "SANLAM LTD" : "Life Groups & Retail Life",

        "STOREBRAND ASA" : "Other insurers", "CNP ASSURANCES" : "Other insurers",
        "GJENSIDIGE FORSI" : "Other insurers", "POSTE ITALIANE" : "Other insurers",
        "NN GROUP" : "Other insurers", "TOPDANMARK A/S" : "Other insurers",
        "BALOISE HOL-REG" : "Other insurers", "SAMPO OYJ-A SHS" : "Other insurers",
        "MAPFRE SA" : "Other insurers", "TRYG A/S" : "Other insurers",
        "ASR NEDERLAND NV" : "Other insurers", "HELVETIA HOL-REG" : "Other insurers",
        "INTACT FINANCIAL" : "Other insurers"
}

yahoo_ticker = {
        "ADMIRAL GROUP" : "ADM.L", "DIRECT LINE INSU" : "DLG.L", 
        "SABRE INSUR" : "SBRE.L", "SAGA PLC" : "SAGA.L",
        "AGEAS" : "AGS.BR", 
        
         "ALLIANZ SE-REG" : "ALV.DE", "AXA" : "CS.PA", 
        "ZURICH INSURANCE" : "ZURN.SW", "GENERALI ASSIC" : "G.MI",

        "BEAZLEY PLC" : "BEZ.L", "HISCOX LTD" : "HSX.L", 
        "LANCASHIRE HOLDI" : "LRE.L",

        "Society of Lloyd’s" : "LLOY.L",

        "AMERICAN INTERNA" : "AIG", "AXIS CAPITAL" : "AXS",
        "TRAVELERS COS IN" : "TRV", "ARCH CAPITAL GRP" : "ACGL",
        "RENAISSANCERE" : "RNR", "EVEREST RE GROUP" : "RE",
        "MARKEL CORP" : "MKL", "HARTFORD FINL SV" : "HIG",
        "ARGO GROUP INTER" : "ARGO", "BERKSHIRE HATH-B" : "BRK-B",
        "CHUBB LTD" : "CB",

        "SCOR SE" : "SCR.PA", "MUENCHENER RUE-R" : "MUV2.DE", 
        "SWISS RE AG" : "SREN.SW", "HANNOVER RUECK S" : "HNR1.DE",

        "TOKIO MARINE HD" : "8766.T", "SOMPO HOLDINGS I" : "8630.T",
        "MS&AD INSURANCE" : "8725.T", "QBE INSURANCE" : "QBE.AX",

        "ENSTAR GROUP LTD" : "ESGR", "FAIRFAX FINL HLD" : "FFH.TO", "RANDALL & QUILTE" : "RQIH.L",

        "PRUDENTIAL PLC" : "PRU.L", "M&G PLC" : "MNG.L",
        "LEGAL & GEN GRP" : "LGEN.L", "AVIVA PLC" : "AV.L",
        "PHOENIX GROUP HO" : "PHNX.L", "QUILTER PLC" : "QLT.L",
        "JUST GROUP" : "JUST.L", "ST JAMES'S PLACE" : "STJ.L",
        "AEGON NV" : "AGN.AS", "DISCOVERY LTD" : "DSY.JO",
        "SANLAM LTD" : "SLM.JO",

        "STOREBRAND ASA" : "STB.OL", "CNP ASSURANCES" : "CNP.PA",
        "GJENSIDIGE FORSI" : "GJF.OL", "POSTE ITALIANE" : "PST.MI",
        "NN GROUP" : "NN.AS", "TOPDANMARK A/S" : "TOP.CO",
        "BALOISE HOL-REG" : "BALN.SW", "SAMPO OYJ-A SHS" : "SAMPO.HE",
        "MAPFRE SA" : "MAP.MC", "TRYG A/S" : "TRYG.CO",
        "ASR NEDERLAND NV" : "0RHS.IL", "HELVETIA HOL-REG" : "HELN.SW",
        "INTACT FINANCIAL" : "IFC.TO"
}

company_name_match['Group'] =company_name_match['Real_Company_Name'].map(Groups)
company_name_match['yfiance_ticker'] =company_name_match['Real_Company_Name'].map(yahoo_ticker)
company_name_match

,Real_Company_Name,company_name,Group,yfiance_ticker
0,ADMIRAL GROUP,Admiral Group,Motor/Personal,ADM.L
1,DIRECT LINE INSU,DIRECT LINE,Motor/Personal,DLG.L
2,SABRE INSUR,SABRE INSUR-,Motor/Personal,SBRE.L
3,SAGA PLC,Saga PLC-,Motor/Personal,SAGA.L
4,AGEAS,Ageas SA-NV-,Motor/Personal,AGS.BR
...,...,...,...,...
62,MAPFRE SA,NaN,Other insurers,MAP.MC
63,TRYG A/S,Tryg A-S-,Other insurers,TRYG.CO
64,ASR NEDERLAND NV,ASR Nederland,Other insurers,0RHS.IL
65,HELVETIA HOL-REG,Helvetia Holding,Other insurers,HELN.SW


In [57]:
df_company = company_name_match[['company_name','Group','yfiance_ticker']]
QA_model_df = df_company.merge(QA_model_df, how='inner', on='company_name')
QA_model_df = QA_model_df.rename(columns={'yfiance_ticker':'ticker'})
QA_model_df

,company_name,Group,ticker,file_name,date,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,WA_sentiment
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.054848,0.158767,0.000649,0.121925,0.072709,0.158536,0.081791,0.001102,0.000962,0.019518,0.077290,0.251904,0.000826
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.002386,0.158865,0.018418,0.034839,0.128776,0.252142,0.182165,0.067381,0.062381,0.027415,0.064643,0.000588,0.000971
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.027649,0.058448,0.001426,0.004682,0.013177,0.135053,0.157797,0.037805,0.446674,0.068490,0.047127,0.001671,0.001851
3,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.000843,0.213648,0.004275,0.046098,0.087957,0.012637,0.069095,0.175282,0.050077,0.005097,0.072873,0.262119,0.001353
4,Admiral Group,Motor/Personal,ADM.L,20130306_Admiral_Group_PLC-_Earnings_Call_2013...,2013-03-06,0.022662,0.750460,0.013020,0.016422,0.020791,0.034012,0.023247,0.024604,0.021481,0.035123,0.022911,0.015267,-0.046780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,0.031631,0.174909,0.010964,0.004023,0.014789,0.234252,0.235555,0.101355,0.109688,0.015269,0.063824,0.003740,0.007517
1366,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,0.018994,0.271337,0.018173,0.051306,0.019724,0.050833,0.339499,0.004691,0.020143,0.040094,0.051085,0.114123,0.003768
1367,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,0.019633,0.178471,0.016324,0.040865,0.060137,0.217649,0.193147,0.027646,0.034454,0.054737,0.100289,0.056647,0.009691
1368,INTACT FINANCIAL-,Other insurers,IFC.TO,20211110_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-11-10,0.019171,0.314577,0.015499,0.036844,0.011301,0.207711,0.160664,0.036279,0.002716,0.011042,0.035831,0.148366,0.000850


In [58]:
start_date='2010-12-01'
end_date='2022-01-30'
stock_list = company_name_match['yfiance_ticker']
df_stock = stock_price_change(stock_list, start_date, end_date)

# limit the dat till 2021-12-31
df_stock = df_stock[df_stock['date']<='2021-12-31']

df_stock


1 Failed download:
- CNP.PA: No data found, symbol may be delisted


,date,Close,ticker,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,2010-12-01,1561.000000,ADM.L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2010-12-02,1589.000000,ADM.L,0.017937,0.010699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2010-12-03,1606.000000,ADM.L,0.010699,0.002491,0.028828,0.005604,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2010-12-06,1610.000000,ADM.L,0.002491,0.003106,0.013216,-0.021739,0.031390,-0.029193,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2010-12-07,1615.000000,ADM.L,0.003106,-0.024768,0.005604,-0.032198,0.016362,-0.036533,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,2021-12-23,163.240005,IFC.TO,-0.001101,0.001470,0.001841,0.008025,0.012090,0.006493,0.005049,0.002940,0.012153,0.010475,0.004678,-0.010843
2777,2021-12-24,163.479996,IFC.TO,0.001470,0.006545,0.000367,0.005016,0.003314,0.005750,0.003684,0.000856,0.013138,0.004037,0.006774,-0.013335
2778,2021-12-29,164.550003,IFC.TO,0.006545,-0.001519,0.008025,-0.000790,0.006915,-0.005044,0.020212,0.011972,0.020022,-0.002492,0.013114,-0.005409
2779,2021-12-30,164.300003,IFC.TO,-0.001519,0.000730,0.005016,-0.003530,0.006493,-0.004139,0.008347,0.003956,0.033984,-0.022581,0.004586,0.000000


In [59]:
df_stock['date'] = df_stock['date'].astype(str)
# merged company tickers into big dataframe
QA_model_df = QA_model_df.join(df_stock.set_index(["date","ticker"]), 
                         on=["date","ticker"],
                         how='left'
                        )
QA_model_df = QA_model_df.dropna()
QA_model_df = QA_model_df.reset_index(drop=True)
QA_model_df

,company_name,Group,ticker,file_name,date,topic_1,topic_2,topic_3,topic_4,topic_5,...,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,Admiral Group,Motor/Personal,ADM.L,20110302_Admiral_Group_PLC-_Earnings_Call_2011...,2011-03-02,0.054848,0.158767,0.000649,0.121925,0.072709,...,-0.020130,0.015106,-0.003612,-0.000604,-0.011350,-0.003021,-0.029326,-0.060423,-0.030463,-0.045317
1,Admiral Group,Motor/Personal,ADM.L,20110824_Admiral_Group_PLC-_Earnings_Call_2011...,2011-08-24,0.002386,0.158865,0.018418,0.034839,0.128776,...,-0.110454,-0.025868,-0.094983,-0.005913,-0.125969,0.011086,-0.031496,0.008130,-0.117417,-0.028825
2,Admiral Group,Motor/Personal,ADM.L,20111109_Admiral_Group_PLC-_Guidance_Call_2011...,2011-11-09,0.027649,0.058448,0.001426,0.004682,0.013177,...,-0.247881,-0.053521,-0.251686,-0.061972,-0.234914,-0.098028,-0.276691,-0.025352,-0.275510,0.039437
3,Admiral Group,Motor/Personal,ADM.L,20120307_Admiral_Group_PLC-_Earnings_Call_2012...,2012-03-07,0.000843,0.213648,0.004275,0.046098,0.087957,...,0.095785,0.012238,0.080264,0.022727,0.062210,0.036713,0.103182,0.020979,0.179990,0.014860
4,Admiral Group,Motor/Personal,ADM.L,20130306_Admiral_Group_PLC-_Earnings_Call_2013...,2013-03-06,0.022662,0.750460,0.013020,0.016422,0.020791,...,0.045455,0.014243,0.069767,0.018741,0.066347,0.006747,0.033308,-0.001499,0.044636,-0.008996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,INTACT FINANCIAL-,Other insurers,IFC.TO,20201104_INTACT_FINANCIAL-_Earnings_Call_2020-...,2020-11-04,0.060781,0.179414,0.015668,0.017713,0.040878,...,0.069258,-0.019425,0.073609,-0.060778,0.063179,-0.080203,0.031990,0.000203,0.026255,-0.019628
1319,INTACT FINANCIAL-,Other insurers,IFC.TO,20210210_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-02-10,0.031631,0.174909,0.010964,0.004023,0.014789,...,0.041432,-0.016994,0.039294,-0.032275,0.050876,-0.041101,0.061678,-0.054802,0.054013,-0.033263
1320,INTACT FINANCIAL-,Other insurers,IFC.TO,20210512_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-05-12,0.018994,0.271337,0.018173,0.051306,0.019724,...,-0.027909,-0.011521,-0.015945,-0.015218,-0.009157,-0.027663,-0.003744,0.002403,-0.002704,0.041341
1321,INTACT FINANCIAL-,Other insurers,IFC.TO,20210728_INTACT_FINANCIAL-_Earnings_Call_2021-...,2021-07-28,0.019633,0.178471,0.016324,0.040865,0.060137,...,0.008669,0.000706,0.007592,-0.002826,0.000294,0.000530,0.001238,0.026725,-0.010427,0.036320


In [60]:
QA_model_df.to_csv("./regression_df_input/QA_model_df.csv", index = False)